In [13]:
import numpy as np
import random
import csv
import copy
import random
from datetime import datetime
import sys

limit_number = 15000
sys.setrecursionlimit(limit_number)

class TspProblem():
    def __init__(self):
        self.cities = []
        self.population = []
        self.total_cost = []
        self.newPop = []
        self.givenSolution = []
        
    def distance(self,x, y): # Euclidean distance
        dist = np.linalg.norm(np.array(x)-np.array(y))
        return dist
    
    #
    def readTSP(self,file): 
        with open(file, mode='r', newline='') as tsp:
            reader = csv.reader(tsp)
            for i in reader:
                self.cities.append(i)
    
    def readSol(self,file): # index of given cities list
        with open(file, mode='r', newline='') as solution:
            reader = csv.reader(solution)
            for i in reader:
                self.givenSolution.append(int(i[0]))

    def genPop(self,size): 
        for i in range(size):
            chromosome = self.givenSolution
            random.shuffle(chromosome)
            indexOfStart = chromosome.index(0) # find start point
            front = chromosome[indexOfStart:]
            back = chromosome[0:indexOfStart]
            chromosome = front + back
            chromosome.append(int(0))
            self.population.append(chromosome)  
        
    def fitnessEval(self):
        fitnessEval = []
        for i in range(len(self.population)):
            distance = [] 
            total_cost = 0
            for idx in range(len(self.givenSolution)):
                pos_city_1 = [float(self.cities[self.population[i][idx]][0]), float(self.cities[self.population[i][idx]][1])]
                pos_city_2 = [float(self.cities[self.population[i][idx+1]][0]), float(self.cities[self.population[i][idx+1]][1])]
                distance.append(self.distance(pos_city_1, pos_city_2)) 
            
            total_cost = sum(distance)
            fitnessEval.append(total_cost) # 총 거리
        return fitnessEval
    
    def select(self,fitnessEval): 
        parents = []
        modFit = []
        for i in fitnessEval:
            modFit.append((SIZEOFCHROMO * 100) - i)
        Idx = modFit.index(max(modFit))
        Idx2 = modFit.index(min(modFit))
        modFit[Idx] *= 2.0
        modFit[Idx2] *= 0.5
        max_value = sum(modFit)
        
        for j in range(2):
            pick = random.uniform(0, max_value)
            current = 0
            for i in modFit:
                current += i
                if current > pick:
                    parents.append(modFit.index(i))
                    break
        return parents
            
        
    def crossOver(self,selectIdx):
        child = [-1 for i in range(len(self.population[selectIdx[0]]))]
        ind = 0
        cross1 = (self.population[selectIdx[0]])
        cross2 = (self.population[selectIdx[1]])
        
        while True:
            # 자식의 값이 모두 채워지면 종료
            if child.count(-1)==0:
                break

            child[ind] = cross1[ind]
            ind = cross1.index(cross2[ind])
            
            if child[ind]!=-1 and child.count(-1)!=0:
                cross3 = cross1
                cross1 = cross2
                cross2 = cross3
                ind = child.index(-1)
        child.pop()
        child = self.mutate(child)
        child.append(child[0])
        
        return  self.newPop.append(child)  

    def findCloseNode(self,chromosome,point):
        distance = [] # distance 초기화
        for idx in range(len(chromosome)):
            pos_city_1 = [float(self.cities[chromosome[point]][0]), float(self.cities[chromosome[point]][1])]
            pos_city_2 = [float(self.cities[chromosome[idx]][0]), float(self.cities[chromosome[idx]][1])]
            
            distance.append(self.distance(pos_city_1, pos_city_2)) # 두 도시의 거리를 스택, 총 1000개가 입력된다.
            if distance[-1] == 0: distance[-1] = 100000
        nearestNode = distance.index(min(distance)) 
        return nearestNode
    
    def mutate(self,chromosome):
        if random.random() < MUTATION_RATE:
            for i in range(MUTATION_TIME):
                point1 = random.randint(1, len(chromosome)-2)
                nearestNode = self.findCloseNode(chromosome,point1)
                if chromosome[nearestNode] == 0: continue
                chromosome[point1+1], chromosome[nearestNode] = chromosome[nearestNode], chromosome[point1+1]
        if random.random()*100 < MUTATION_RATE:
            for i in range(MUTATION_TIME):
                print("이벤트 돌연변이")
                point1 = random.randint(1, len(chromosome)-2)
                point2 = random.randint(1, len(chromosome)-2)
                chromosome[point1], chromosome[point2] = chromosome[point2], chromosome[point1]
            
        return chromosome

    def changeGen(self):
        self.population = self.newPop.copy()
        self.newPop = []
    

  
f = open('/home/code/Jo/stdout4.txt', 'w')
SIZEOFCHROMO = 1000
INITPOPSIZE = 50
MUTATION_RATE = 0.01
MUTATION_TIME = 3
POPSIZE = 50
GENERATION = 1000
a = TspProblem()
a.readTSP('/home/code/Jo/TSP.csv')
a.readSol('/home/code/Jo/example_solution.csv')
a.genPop(INITPOPSIZE)                               
print("{0}, {1}, {2}, {3}".format(SIZEOFCHROMO,MUTATION_RATE,MUTATION_TIME,POPSIZE), file = f)
for i in range(GENERATION):
    fitness = a.fitnessEval()
    for j in range(POPSIZE):
        parentsIdx = a.select(fitness)
        a.crossOver(parentsIdx)

    a.changeGen()
    minimum = min(fitness)    
    print("{0}, {1}".format(i,minimum))
    print("{0}, {1}".format(i,minimum), file = f)

print("{0}세대, 최소 비용은 {1} ".format(i,minimum), file = f)
fitIdx = fitness.index(min(fitness))
print(a.population[fitIdx], file = f)
f.close()

[0, 661, 855, 614, 183, 347, 561, 426, 639, 346, 553, 425, 819, 24, 4, 880, 138, 931, 963, 112, 413, 248, 911, 910, 25, 146, 581, 755, 69, 511, 676, 220, 835, 42, 195, 921, 912, 34, 222, 319, 827, 411, 493, 892, 585, 278, 990, 501, 841, 702, 944, 737, 261, 942, 198, 257, 99, 77, 394, 727, 772, 515, 276, 144, 216, 266, 810, 777, 618, 79, 988, 402, 824, 613, 699, 455, 108, 848, 664, 437, 247, 32, 341, 167, 860, 107, 964, 194, 551, 809, 30, 807, 166, 711, 363, 212, 663, 902, 932, 134, 476, 825, 111, 324, 116, 584, 274, 537, 475, 354, 630, 681, 719, 650, 994, 603, 758, 527, 811, 230, 793, 689, 782, 640, 563, 281, 593, 850, 262, 38, 672, 602, 956, 529, 965, 237, 488, 608, 512, 580, 218, 429, 623, 139, 359, 842, 526, 692, 497, 802, 83, 434, 245, 292, 535, 730, 80, 157, 321, 565, 252, 301, 214, 770, 879, 410, 741, 939, 143, 123, 759, 294, 620, 936, 94, 21, 430, 272, 190, 156, 731, 586, 208, 129, 983, 331, 400, 991, 829, 391, 579, 433, 473, 88, 2, 508, 694, 467, 503, 583, 691, 509, 778, 383, 6